In [ ]:
try:
    from ase import Atoms
    from ase.calculators.lj import LennardJones
    print("successfully imported ase")
except ImportError:
    !! pip install ase --user --upgrade
    print("completed installing ASE")

In [ ]:
try:
    import torch
    print("successfully imported torch")
except ImportError:
    !! pip install torch --user --upgrade
    print("completed installing torch")

In [ ]:
try:
    import dscribe
    print("successfully imported dscribe")
except ImportError:
    !! pip install dscribe --user --upgrade
    print("completed installing dscribe")

In [ ]:
try:
    import numpy as np
    print("successfully imported numpy")
except ImportError:
    !! pip install numpy --user --upgrade
    print("completed installing numpy")

In [ ]:
try:
    from matplotlib import pyplot as plt
    print("successfully imported matplotlib")
except ImportError:
    !! pip install matplotlib --user --upgrade
    print("completed installing matplotlib")

In [ ]:
!! pip install --user --upgrade aenet-gpr
print("completed installing aenet-gpr")

import aenet_gpr
print(aenet_gpr.__version__)

from aenet_gpr.util import ReferenceData
from aenet_gpr.src import gpr_batch
from aenet_gpr.inout import io_train_parameters, io_test_parameters
from aenet_gpr.util.prepare_data import standard_output, inverse_standard_output

# 1. Read reference training/test data from xsf files

We are using train/test data of H-H Lennard-Jones potential prepared in `aenet-gpr/example/1_H2/`

In [ ]:
import os.path

if os.path.isdir("./1_H2/"):
    ! mkdir 1_H2
    
if os.path.isfile("../example/1_H2/train_set.zip"):
    ! unzip -oq ../example/1_H2/train_set.zip -d ./1_H2/
else:
    ! wget https://github.com/atomisticnet/aenet-gpr/raw/refs/heads/main/example/1_H2/train_set.zip
    ! unzip -oq train_set.zip -d ./1_H2/
    ! rm train_set.zip

In [ ]:
if os.path.isfile("../example/1_H2/test_set.zip"):
    ! unzip -oq ../example/1_H2/test_set.zip -d ./1_H2/
else:
    ! wget https://github.com/atomisticnet/aenet-gpr/raw/refs/heads/main/example/1_H2/test_set.zip
    ! unzip -oq test_set.zip -d ./1_H2/
    ! rm test_set.zip

First, let aenet-gpr read the train/test files

In [ ]:
import glob


train_xsf_files = glob.glob("./1_H2/train_set/file_*")
train_xsf_files.sort()
train_data = ReferenceData(structure_files=train_xsf_files, file_format='xsf')
train_data.set_data()

print(train_data.energy.shape)
print(train_data.force.shape)

In [ ]:
test_xsf_files = glob.glob("./1_H2/test_set/file_*")
test_xsf_files.sort()
test_data = ReferenceData(structure_files=test_xsf_files, file_format='xsf')
test_data.set_data()

print(test_data.energy.shape)
print(test_data.force.shape)

# 2. Model train

Second, construct a GPR model fitted to training energy and force data (7 sample points). 

In this example, default settings, energy/force standardization and sqexp kernel parameters, are used.

In [ ]:
if train_data.standardization:
    train_data.standardize_energy_force(train_data.energy)
train_data.config_calculator()

io_train_parameters(data_param=train_data.write_params())

# 3. Model evaluation for test set

Finally, using the constructed GPR model, evaluate energy and force of test structures (200 structures).

In [ ]:
test_data.calculator = train_data.calculator

io_test_parameters(data_param=test_data.write_params())

energy_test_gpr, force_test_gpr, uncertainty_test_gpr = test_data.evaluation(get_variance=True)

Since the training energy/force values were standardized, the raw predictions should also be standardized.

Final predictions on energy and force can be obtained from inverse standardization.

In [ ]:
if train_data.standardization:
    energy_test_gpr, force_test_gpr = inverse_standard_output(energy_ref=train_data.energy,
                                                              scaled_energy_target=energy_test_gpr,
                                                              scaled_force_target=force_test_gpr)

abs_force_test_gpr = np.linalg.norm(force_test_gpr, axis=2)
abs_force_test = np.linalg.norm(test_data.force, axis=2)

print("Energy MAE (eV):", np.absolute(np.subtract(energy_test_gpr, test_data.energy)).mean())
print("Force MAE (eV/Ang):", np.absolute(np.subtract(abs_force_test_gpr, abs_force_test)).mean())

Let's check the GPR energy and force predictions in comparison with Lennard-Jones potential.

The results will be shown by energy/force vs H-H bond distance plots.

In [ ]:
d_train = []
for image in train_data.images:
    d_train.append(np.linalg.norm(image.get_positions()[1] - image.get_positions()[0]))

d_test = []
for image in test_data.images:
    d_test.append(np.linalg.norm(image.get_positions()[1] - image.get_positions()[0]))

In [ ]:
fig = plt.figure(figsize=(3.5, 5.5))
ax1, ax2, ax3 = fig.subplots(3, 1, height_ratios=[1, 1, 1.5], sharex=False, sharey=False)

font_x = {'size': 16, 'color': 'black'}
font_y = {'size': 16, 'color': 'black'}
font_tick = {'size': 12, 'color': 'black'} 


# The 1st plot
ax1.fill_between(d_test, uncertainty_test_gpr, 0, label='Uncertainty', color='crimson', alpha=0.5, edgecolor='black')
ax1.vlines(d_train, ymin=-100, ymax=100, color='black', linestyle='--', linewidth=1)

# ax1.set_xlabel("Error in energy [eV]", fontdict=font_x)
ax1.set_xlim([0.95, 2.05])
x_labels = [round(label, 2) for label in ax1.get_xticks()]
ax1.set_xticks(x_labels)
ax1.set_xlim([0.95, 2.05])
ax1.set_xticklabels(x_labels, fontdict=font_tick)

ax1.set_ylabel('Uncertainty', fontdict=font_y)
ax1.set_ylim(0, 0.05)
y_labels = [round(label, 3) for label in ax1.get_yticks()]
y_labels = [0.01, 0.03, 0.05]
ax1.set_yticks(y_labels)
ax1.set_ylim([0, 0.05])
ax1.set_yticklabels(y_labels, fontdict=font_tick)

[x.set_linewidth(1.5) for x in ax1.spines.values()]
ax1.tick_params(bottom=True, top=True, left=True, right=True)
ax1.tick_params(labelbottom=False, labeltop=False, labelleft=True, labelright=False)
ax1.tick_params(direction='in', length=8, width=1.5)


# The second plot
ax2.fill_between(d_test, np.subtract(energy_test_gpr, test_data.energy), 0, color='crimson', alpha=0.5, edgecolor='black')
ax2.vlines(d_train, ymin=-100, ymax=100, color='black', linestyle='--', linewidth=1)

ax2.set_xlim([0.95, 2.05])
x_labels = [round(label, 2) for label in ax2.get_xticks()]
ax2.set_xticks(x_labels)
ax2.set_xlim([0.95, 2.05])
ax2.set_xticklabels(x_labels, fontdict=font_tick)

ax2.set_ylabel("Error (eV)", fontdict=font_y)
ax2.set_ylim([-0.05, 0.05])
y_labels = [round(label, 2) for label in ax2.get_yticks()]
y_labels = [-0.04, -0.02, 0.0, 0.02, 0.04]
ax2.set_yticks(y_labels)
ax2.set_ylim([-0.05, 0.05])
ax2.set_yticklabels(y_labels, fontdict=font_tick)

[x.set_linewidth(1.5) for x in ax2.spines.values()]
ax2.tick_params(bottom=True, top=True, left=True, right=True)
ax2.tick_params(labelbottom=False, labeltop=False, labelleft=True, labelright=False)
ax2.tick_params(direction='in', length=8, width=1.5)


# The third plot
ax3.plot(d_test, test_data.energy, '-', color='gray', label='Target', alpha=0.5, linewidth=10)
ax3.plot(d_test, energy_test_gpr, '-', label='GPR', color='crimson', linewidth=4)
ax3.vlines(d_train, ymin=-100, ymax=100, color='black', linestyle='--', linewidth=1)

ax3.plot(d_train, train_data.energy, 'o', label='Data', color='black', markersize=8)

ax3.set_xlabel('H-H distance (Ang)', fontdict=font_x)
ax3.set_xlim(0.95, 2.05)
x_labels = [round(label, 2) for label in ax3.get_xticks()]
ax3.set_xticks(x_labels)
ax3.set_xticklabels(x_labels, fontdict=font_tick)
ax3.set_xlim(0.95, 2.05)

ax3.set_ylabel('Energy (eV)', fontdict=font_y)
ax3.set_ylim(-1.4, 2.2)
y_labels = [round(label, 2) for label in ax3.get_yticks()]
# y_labels = [-1.0, -0.5, 0.0, 0.5, 1.0, 1.5, 2.0]
ax3.set_yticks(y_labels)
ax3.set_yticklabels(y_labels, fontdict=font_tick)
ax3.set_ylim(-1.4, 2.2)

ax3.legend(loc='upper right', fontsize=12, ncol=1, frameon=True)
[x.set_linewidth(1.5) for x in ax3.spines.values()]
ax3.tick_params(bottom=True, top=True, left=True, right=False)
ax3.tick_params(labelbottom=True, labeltop=False, labelleft=True, labelright=False)
ax3.tick_params(direction='in', length=8, width=1.5)

fig.tight_layout()
plt.subplots_adjust(hspace=0.05, wspace=0.05)
plt.show()

In [ ]:
fig = plt.figure(figsize=(3.5, 4.5))
ax2, ax3 = fig.subplots(2, 1, height_ratios=[1, 1.5], sharex=False, sharey=False)

font_x = {'size': 16, 'color': 'black'}
font_y = {'size': 16, 'color': 'black'}
font_tick = {'size': 12, 'color': 'black'}


# The second plot
ax2.fill_between(d_test, np.subtract(force_test_gpr[:, 0, 0], test_data.force[:, 0, 0]), 0, color='crimson', alpha=0.5, edgecolor='black')
ax2.vlines(d_train, ymin=-100, ymax=100, color='black', linestyle='--', linewidth=1)

ax2.set_xlim([0.95, 2.05])
x_labels = [round(label, 2) for label in ax2.get_xticks()]
ax2.set_xticks(x_labels)
ax2.set_xlim([0.95, 2.05])
ax2.set_xticklabels(x_labels, fontdict=font_tick)

ax2.set_ylabel("Error (eV/Ang)", fontdict=font_y)
ax2.set_ylim([-1.2, 1.2])
y_labels = [round(label, 2) for label in ax2.get_yticks()]
y_labels = [-1.0, -0.5, 0.0, 0.5, 1.0]
ax2.set_yticks(y_labels)
ax2.set_ylim([-1.2, 1.2])
ax2.set_yticklabels(y_labels, fontdict=font_tick)

[x.set_linewidth(1.5) for x in ax2.spines.values()]
ax2.tick_params(bottom=True, top=True, left=True, right=True)
ax2.tick_params(labelbottom=False, labeltop=False, labelleft=True, labelright=False)
ax2.tick_params(direction='in', length=8, width=1.5)


# The third plot
ax3.plot(d_test, test_data.force[:, 0, 0], '-', color='gray', alpha=0.5, linewidth=10)
ax3.plot(d_test, force_test_gpr[:, 0, 0], '-', label='GPR (PyTorch)', color='crimson', linewidth=4)
ax3.vlines(d_train, ymin=-100, ymax=100, color='black', linestyle='--', linewidth=1)

ax3.plot(d_train, train_data.force[:, 0, 0], 'o', label='Original F', color='black', markersize=8)

ax3.set_xlabel('H-H distance (Ang)', fontdict=font_x)
ax3.set_xlim(0.95, 2.05)
x_labels = [round(label, 2) for label in ax3.get_xticks()]
ax3.set_xticks(x_labels)
ax3.set_xticklabels(x_labels, fontdict=font_tick)
ax3.set_xlim(0.95, 2.05)

ax3.set_ylabel('Force (eV/Ang)', fontdict=font_y)
ax3.set_ylim(-40, 5)
y_labels = [int(label) for label in ax3.get_yticks()]
# y_labels = [-40, -30, -20, -10, 0]
ax3.set_yticks(y_labels)
ax3.set_yticklabels(y_labels, fontdict=font_tick)
ax3.set_ylim(-40, 5)

# ax3.legend(loc='lower right', fontsize=12, ncol=1, frameon=True)
[x.set_linewidth(1.5) for x in ax3.spines.values()]
ax3.tick_params(bottom=True, top=True, left=True, right=False)
ax3.tick_params(labelbottom=True, labeltop=False, labelleft=True, labelright=False)
ax3.tick_params(direction='in', length=8, width=1.5)

fig.tight_layout()
plt.subplots_adjust(hspace=0.05, wspace=0.05)

plt.show()